In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import pymysql as mdb
%matplotlib inline
import geocoder
import seaborn as sns
from bs4 import BeautifulSoup
import urllib 
import pickle
import pymysql as mdb

/usr/local/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


## scrape outdoor pool data and make a table with pool, address, coordinates, and dimensions

In [231]:
r = urllib.urlopen('https://www.nycgovparks.org/facilities/outdoor-pools').read()
soup = BeautifulSoup(r, "lxml")

In [232]:
#take a look to see what to call
print soup.prettify()[15000:]
#looks like i want to find three things, div class = poolbox, poolbox even and poolbox even zstripe

7:00 p.m.
       </strong>
       , with a break for pool cleaning between 3:00 p.m. and 4:00 p.m. Early bird and night owl
       <a href="http://www.nycgovparks.org/registration/lapswim">
        lap swim hours
       </a>
       are also available at select sites.
      </p>
      <p class="span4 pull-right">
       <img alt="person swimming laps" height="100" src="/facilities/images/outdoor-pools-swim-programs-feat.jpg" width="254"/>
      </p>
      <h2>
       Free Swim Programs
      </h2>
      <p>
       Find out more about our learn to swim program, our youth swim team, and programs for seniors and people with disabilities.
      </p>
      <p class="more">
       <a href="/programs/aquatics">
        Learn more about free swim programs
       </a>
      </p>
      <p>
       Looking for our free Learn to Swim program? Find out more by visiting our
       <a href="/registration/aquatics">
        Learn to Swim
       </a>
       registration page.
      </p>
      <h2>
      

In [233]:
#class poolbox finds all kinds
poolbox = soup.find_all("div", class_="poolbox")
#poolboxeven = soup.find_all("div", class_="poolbox even")
#poolboxevenz = soup.find_all("div", class_="poolbox even zstripe")
#poolboxz = soup.find_all("div", class_="poolbox zstripe")

In [71]:
pools = []
address = []
dimensions = []
for i, j in enumerate(poolbox):
    pools.append(re.findall(r"<h3>(.+)</h3>", str(j)))
    pools[i].append(re.findall(r"</h3>(.+)<br/> <!-", str(j)))
    pools[i].append(re.findall(r"Dimensions: (.+)'</li>", str(j)))
   

In [139]:
pools
#will haveto fix dimensions where there are multiple outdoor pools, smaller pools are always second
#so try to just get first

[['Abe Lincoln Pool', ['5th Avenue and E. 135th Street'], ["40' x 20' x 3"]],
 ['Asser Levy Pool',
  ['Asser Levy Place and E. 23rd Street'],
  ["120' x 45' x 4'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 41.67' x 25' x 1.5"]],
 ['Dry Dock Pool',
  ['E. 10th St. between Aves. C and D'],
  ["75' x 60' x 3'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 30' x 20' x 1.5"]],
 ['Frederick Douglass Pool',
  ['Amsterdam Ave. between 100th and 102nd Streets'],
  ["40' x 20' x 3"]],
 ['Hamilton Fish Pool',
  ['Pitt Street and Houston Street'],
  ["165' x 98' x 3.5'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 67' x 98' x 2.5"]],
 ['Highbridge Pool',
  ['Amsterdam Ave. and W. 173rd Street'],
  ["220' x 162' x 10.5'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 220' x 92' x 2"]],
 ['Jackie Robinson Pool',
  ['Bradhurst Avenue and W. 146th Street'],
  ["235' x 82' x 3.5"]],
 ['John Jay Pool',
  ['East of York Avenue on 77th St.'],
  ["

In [237]:
poolbox

[<div class="poolbox even zstripe"> <h3>Abe Lincoln Pool</h3>5th Avenue &amp; E. 135th Street<br/> <!--(212) 491-1714<br />--> <ul class="unstyled"><li><strong>Outdoor Mini Pool</strong><br/>Dimensions: 40' x 20' x 3'</li></ul> <p class="more"><a href="/parks/M193/facilities/outdoor-pools">Abe Lincoln Pool Details</a></p> </div>,
 <div class="poolbox"> <h3>Asser Levy Pool</h3>Asser Levy Place and E. 23rd Street<br/> <!--(212) 447-2020<br />--> <ul class="unstyled"><li><img alt="This is an Accessible Pool" class="pull-left" height="25" src="/common_files/uploaded_content/park_rules_images/image-16931.gif" width="25"/><strong>Outdoor Intermediate Pool</strong><br/>Dimensions: 120' x 45' x 4'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 41.67' x 25' x 1.5'</li></ul> <p class="more"><a href="/parks/M164/facilities/outdoor-pools">Asser Levy Pool Details</a></p> </div>,
 <div class="poolbox even"> <h3>Dry Dock Pool</h3>E. 10th St. between Aves. C and D<br/> <!--(212) 677-4481

In [296]:
#retrying to get dimensions

outdoordimensions = []
for i, j in enumerate(poolbox):
    
    outdoordimensions.append(re.findall(r"<h3>(.+)</h3>", str(j)))
    #pools[i].append(re.findall(r"</h3>(.+)<br/> <!-", str(j)))
    outdoordimensions[i].append(re.findall("Dimensions: ([a-z0-9' .]*)</li>", str(j)))


In [302]:
#try with dict instead
#retrying to get dimensions

outdoordict = {}
for j in poolbox:
    try:
        outdoordict[re.findall(r"<h3>(.+)</h3>", str(j))[0]] = re.findall("Dimensions: ([a-z0-9' .]*)</li>", str(j))[0]
    #pools[i].append(re.findall(r"</h3>(.+)<br/> <!-", str(j)))
    #outdoordimensions[i].append(re.findall("Dimensions: ([a-z0-9' .]*)</li>", str(j)))
    except:
        print re.findall(r"<h3>(.+)</h3>", str(j))[0]

JHS 57/HS 26 Pool


In [308]:
outdoordict
#pickle to save
pickle.dump( outdoordict, open("outdoorpoolanddimensions.p", "wb" ) )

In [306]:
poolist = list(poolcoord.keys())
poolist
newoutdoordim = {}
#want dimensions only for non-mini pools
for i in poolist:
    newoutdoordim[i]= outdoordict[i]

In [313]:
poolcoord

{'Asser Levy Pool': (40.7359523, -73.976311),
 'Astoria Pool': (40.7784602, -73.9213924),
 'Betsy Head Pool': (40.6630185, -73.9121419),
 'Bushwick Pool': (40.701019, -73.94087200000001),
 'Claremont Pool': (40.8378905, -73.911385),
 'Commodore Barry Pool': (40.7674987, -73.833079),
 'Crotona Pool': (40.8406948, -73.8982441),
 'Dry Dock Pool': (40.725358, -73.975523),
 'Faber Pool': (40.6402662, -74.135133),
 'Fisher Pool': (40.7590771, -73.8698387),
 'Floating Pool': (40.8061283, -73.8891529),
 'Fort Totten Pool': (40.7915979, -73.7748836),
 'Haffen Pool': (40.8749139, -73.8383513),
 'Hamilton Fish Pool': (40.72077530000001, -73.9812832),
 'Highbridge Pool': (40.8429613, -73.9346667),
 'Howard Pool': (40.672697, -73.908868),
 'Jackie Robinson Pool': (40.8239331, -73.9424159),
 'John Jay Pool': (40.7699381, -73.949576),
 'Kosciuszko Pool': (40.691323, -73.9485458),
 'Lasker Pool': (40.7982211, -73.95240249999999),
 'Liberty Pool': (40.7020409, -73.78312540000002),
 'Mapes Pool': (40.84

In [314]:

#now just have to attach the coordinates have dictionary called poolcoord with pools and coordinates
#just iterate through and add dimensions after coordinates
outdimcoord = {}
for i in poolist:
    outdimcoord[i] = (poolcoord[i], newoutdoordim[i])

In [315]:
outdimcoord

{'Asser Levy Pool': ((40.7359523, -73.976311), "120' x 45' x 4'"),
 'Astoria Pool': ((40.7784602, -73.9213924), "330' x 165' x 4'"),
 'Betsy Head Pool': ((40.6630185, -73.9121419), "330' x 165' x 4.25"),
 'Bushwick Pool': ((40.701019, -73.94087200000001), "75' x 60' x 3'"),
 'Claremont Pool': ((40.8378905, -73.911385), "75' x 60' x 3.5'"),
 'Commodore Barry Pool': ((40.7674987, -73.833079), "75' x 60' x 3'"),
 'Crotona Pool': ((40.8406948, -73.8982441), "330' x 120' x 4'"),
 'Dry Dock Pool': ((40.725358, -73.975523), "75' x 60' x 3'"),
 'Faber Pool': ((40.6402662, -74.135133), "140' x 76' x 5'"),
 'Fisher Pool': ((40.7590771, -73.8698387), "75' x 60' x 3.5'"),
 'Floating Pool': ((40.8061283, -73.8891529), "82' x 52' x 4'"),
 'Fort Totten Pool': ((40.7915979, -73.7748836), "75' x 40' x 4'"),
 'Haffen Pool': ((40.8749139, -73.8383513), "75' x 60' x 3.5'"),
 'Hamilton Fish Pool': ((40.72077530000001, -73.9812832), "165' x 98' x 3.5'"),
 'Highbridge Pool': ((40.8429613, -73.9346667), "220'

In [140]:
pools[2][2][0]

"75' x 60' x 3'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 30' x 20' x 1.5"

In [73]:
for i in range(0, len(pools)):
    try:
        pools[i][1][0] = str.replace(pools[i][1][0], "&amp;", "and")
    except:
        print i

In [74]:
pools
#these addresses are useless basically

[['Abe Lincoln Pool', ['5th Avenue and E. 135th Street'], ["40' x 20' x 3"]],
 ['Asser Levy Pool',
  ['Asser Levy Place and E. 23rd Street'],
  ["120' x 45' x 4'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 41.67' x 25' x 1.5"]],
 ['Dry Dock Pool',
  ['E. 10th St. between Aves. C and D'],
  ["75' x 60' x 3'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 30' x 20' x 1.5"]],
 ['Frederick Douglass Pool',
  ['Amsterdam Ave. between 100th and 102nd Streets'],
  ["40' x 20' x 3"]],
 ['Hamilton Fish Pool',
  ['Pitt Street and Houston Street'],
  ["165' x 98' x 3.5'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 67' x 98' x 2.5"]],
 ['Highbridge Pool',
  ['Amsterdam Ave. and W. 173rd Street'],
  ["220' x 162' x 10.5'</li><li><strong>Outdoor Wading Pool</strong><br/>Dimensions: 220' x 92' x 2"]],
 ['Jackie Robinson Pool',
  ['Bradhurst Avenue and W. 146th Street'],
  ["235' x 82' x 3.5"]],
 ['John Jay Pool',
  ['East of York Avenue on 77th St.'],
  ["

In [104]:

poolcoord = {}
failpool = []
for i in range(0, len(pools)):
    try:
        g = geocoder.google(pools[i][1][0])
        lon=g.geojson['geometry']['coordinates'][0]
        lat=g.geojson['geometry']['coordinates'][1]
        if lon < -74.257061 or lon > -73.699273 or lat > 40.915087 or lat < 40.498294:
            failpool.append(([pools[i][0]], pools[i][1][0]))
        else:
            poolcoord[pools[i][0]] = (lat, lon)
    except:
        failpool.append(([pools[i][0]], pools[i][1][0]))

In [156]:
poolcoord

{'Abe Lincoln Pool': (40.8127697, -73.9376495),
 'Asser Levy Pool': (40.7359523, -73.976311),
 'Astoria Pool': (40.7784602, -73.9213924),
 'Bronx River Playground Pool': (40.8344546, -73.8792959),
 'Bushwick Pool': (40.701019, -73.94087200000001),
 'Commodore Barry Pool': (40.7674987, -73.833079),
 'Crotona Pool': (40.8406948, -73.8982441),
 'David Fox/PS 251 Pool': (40.6330832, -73.9244842),
 'Dry Dock Pool': (40.725358, -73.975523),
 'Edenwald Houses Pool': (40.8850354, -73.84314309999999),
 'Faber Pool': (40.6402662, -74.135133),
 'Fisher Pool': (40.7590771, -73.8698387),
 'Floating Pool': (40.8061283, -73.8891529),
 'General Douglas Pool': (40.5854331, -74.1014783),
 'Glenwood Houses Pool': (40.6374911, -73.91916549999999),
 'Haffen Pool': (40.8749139, -73.8383513),
 'Hamilton Fish Pool': (40.72077530000001, -73.9812832),
 'Highbridge Pool': (40.8429613, -73.9346667),
 'JHS 57/HS 26 Pool': (40.7422408, -74.23029799999999),
 'Jackie Robinson Pool': (40.8239331, -73.9424159),
 'John 

In [148]:
failpool
#not going to include mini pools so not including tompkins square pool, vesuvio,
#frederick douglass, windmuller pool, brooklyn bridge park pop up, lindower park pool
#ps 20 playground pool, ps 14 playground pool

[(['Dry Dock Pool'], 'E. 10th St. between Aves. C and D'),
 (['Frederick Douglass Pool'],
  'Amsterdam Ave. between 100th and 102nd Streets'),
 (['John Jay Pool'], 'East of York Avenue on 77th St.'),
 (['Tompkins Square Pool'], 'Ave. A between E. 7th and E. 10th Streets'),
 (['Vesuvio Pool'], 'Thompson St. between Spring and Prince Streets'),
 (['Wagner Pool'], 'E. 124th Street between First and Second Avenues'),
 (['Fort Totten Pool'], '338 Story Avenue'),
 (['Windmuller Pool'], '54th Street and 39th Road'),
 (['Betsy Head Pool'], 'Boyland, Livonia and Dumont Avenues'),
 (['Brooklyn Bridge Park Pop-Up Pool'], 'Pier 2'),
 (['Douglas and DeGraw Pool'], 'Third Avenue and Nevins Street'),
 (['Howard Pool'], 'Glenmore and Mother Gaston Blvd., East New York Avenue'),
 (['Kosciuszko Pool'],
  'Marcy Avenue between DeKalb Avenue and Kosciuszko Street'),
 (['Lindower Park Pool'], 'E. 60th St, Mill Rd. and Strickland Avenue'),
 (['PS 20 Playground Pool'], 'Between Clermont Ave. and Adelphi St.'

In [204]:
g = geocoder.google('899 Henderson Ave, Staten Island, NY 10310')
lon=g.geojson['geometry']['coordinates'][0]
lat=g.geojson['geometry']['coordinates'][1]
print lat, lon

40.6365967 -74.1206632


In [205]:
poolcoord['West Brighton Pool'] = (lat, lon)


In [207]:
#delete all mini pools
del poolcoord['Bronx River Playground Pool']
del poolcoord['Edenwald Houses Pool']
del poolcoord['David Fox/PS 251 Pool']
del poolcoord['Glenwood Houses Pool']
del poolcoord['JHS 57/HS 26 Pool']
del poolcoord['Abe Lincoln Pool']
del poolcoord['PS 10 Pool']
del poolcoord['PS 186 Playground Pool']
del poolcoord['General Douglas Pool']
del poolcoord["Mariner's Harbor Playground Pool"]
del poolcoord['PS 46 Playground Pool']


In [210]:
poolcoord
#convert to dataframe and then csv to load into mapbox
df=pd.DataFrame(poolcoord.items(), columns=['pool', 'coordinates'])

In [219]:
df['coordinates'][0][0]

40.7294663

In [221]:
df
f = lambda x:  x[0]
df['latitude']= df['coordinates'].map(f)

In [223]:
f = lambda x:  x[1]
df['longitude']= df['coordinates'].map(f)

In [227]:
del df['coordinates']

In [228]:
df

,pool,latitude,longitude
0,Tony Dapolito Pool,40.729466,-74.005169
1,Mullaly Pool,40.831976,-73.923529
2,Haffen Pool,40.874914,-73.838351
3,Howard Pool,40.672697,-73.908868
4,Wagner Pool,40.803032,-73.936328
5,Tottenville Pool,40.506204,-74.234891
6,Hamilton Fish Pool,40.720775,-73.981283
7,Crotona Pool,40.840695,-73.898244
8,Astoria Pool,40.778460,-73.921392
9,Betsy Head Pool,40.663018,-73.912142


In [230]:
df.to_csv('outdoorpoolcoord2.csv')
#what i really will want is to also include the dimensions...this will take some time but then
#i can have the description on the map is the pool dimensions which will show up when you hover
#already have indoor pool dimensions, so have to add this to the csv.

In [317]:
dfdimcoord=pd.DataFrame(outdimcoord.items(), columns=['pool', 'dimcoord'])

In [320]:
dfdimcoord
dfdimcoord=pd.DataFrame(outdimcoord.items(), columns=['pool', 'dimcoord'])
f = lambda x:  x[0][0]
dfdimcoord['latitude']= dfdimcoord['dimcoord'].map(f)

In [322]:
dfdimcoord
f = lambda x:  x[0][1]
dfdimcoord['longitude']= dfdimcoord['dimcoord'].map(f)

In [324]:
dfdimcoord
f = lambda x:  x[1]
dfdimcoord['dimensions']= dfdimcoord['dimcoord'].map(f)

In [326]:

#save this!!! but first remove the tuple
del dfdimcoord['dimcoord']
dfdimcoord.to_csv('outdoorpooldimcoord.csv')
dfdimcoord
#make a csv with the same columns for indoor pools

,pool,latitude,longitude,dimensions
0,Tony Dapolito Pool,40.729466,-74.005169,100' x 50' x 9'
1,Mullaly Pool,40.831976,-73.923529,75' x 60' x 3.5'
2,Haffen Pool,40.874914,-73.838351,75' x 60' x 3.5'
3,Howard Pool,40.672697,-73.908868,75' x 60' x 3'
4,Wagner Pool,40.803032,-73.936328,75' x 60' x 3.5'
5,Tottenville Pool,40.506204,-74.234891,75' x 60' x 3.5'
6,Hamilton Fish Pool,40.720775,-73.981283,165' x 98' x 3.5'
7,Crotona Pool,40.840695,-73.898244,330' x 120' x 4'
8,Astoria Pool,40.778460,-73.921392,330' x 165' x 4'
9,Betsy Head Pool,40.663018,-73.912142,330' x 165' x 4.25


In [209]:
#indoor pools

In [138]:
#do the same for indoor pools to include dimensions
r = urllib.urlopen('https://www.nycgovparks.org/facilities/indoor-pools').read()
soup = BeautifulSoup(r, "lxml")

In [215]:
print soup.prettify()[15000:]
#indoor and outdoor pool sites are different. here i want h4 for pool name and dimenions and li for dimensions
#furthermore, no obvious div class to use instead use <p class="more-left">

e  to attend free aquatics programs  such as Learn to Swim. Your membership will  give you access to use  every recreation center in the city, including  all 12 indoor pools.
      </p>
      <h2>
       Becoming a Member
      </h2>
      <p>
       Annual memberships for recreation centers with pools cost $150 for  adults, and $25 for seniors (ages 62 and older). Membership is free for  anyone under the age of 18. As of July 1, 2013, memberships will cost  $25 for young adults (ages 18-24).
      </p>
      <p class="more">
       <a href="http://www.nycgovparks.org/programs/recreation-centers/membership">
        Learn more
       </a>
      </p>
      <h2>
       Find a Recreation Center with a Pool
      </h2>
      <p>
       Take a look at the pools below to find out which recreation centers have pools, and to find indoor pool schedules.
      </p>
      <p>
       <img alt="Handicap Accessible" src="/common_files/uploaded_content/park_rules_images/image-16931.gif"/>
       <str

In [164]:
import urllib
import re
source = urllib.urlopen('https://www.nycgovparks.org/facilities/indoor-pools')
page = source.readlines()

In [270]:
indoorpools = []
for i in page:
    pools = re.findall("<h4>([A-Za-z 0-9'.&]*)</h4>", str(i))
    if len(pools) > 1:
        indoorpools.append(pools)
        


In [271]:
indoorpools


[['Asser Levy Pool',
  'Chelsea Pool',
  'Gertrude Ederle Pool',
  'Hansborough Pool',
  'Recreation Center 54 Pool',
  'Tony Dapolito Pool',
  'Brownsville Pool',
  'Metropolitan Pool',
  "St. John's Pool",
  'Flushing Meadows Corona Park Pool & Rink',
  'Roy Wilkins Pool',
  "St. Mary's Pool"]]

In [272]:
indoordimensions = []
for i in page:
    dimensions = re.findall("Dimensions: ([a-z0-9' .]*)</li>", str(i))
    if len(dimensions) > 1:
        indoordimensions.append(dimensions)
        


In [273]:
indoordimensions

[["66' x 26' x 8'",
  "75' x 43' x 10'",
  "60' x 35' x 8'",
  "75' x 35' x 9'",
  "54' x 50' x 12'",
  "70' x 21' x 8'",
  "75' x 30' x 8'",
  "75'x30'",
  "75' x 42' x 9'",
  "164' x 82' x 7.5'",
  "75' x 30' x 9'",
  "72' x 39' x 9'"]]

In [274]:
poolanddim = []
pooldimdict = {}
count = 0
indoorpools2 = indoorpools[0]
indoordimensions2 = indoordimensions[0]
for i,j in enumerate(indoorpools2):
    poolanddim.append((j, indoordimensions2[i]))
    pooldimdict[j] = indoordimensions2[i]

In [276]:
pooldimdict

{'Asser Levy Pool': "66' x 26' x 8'",
 'Brownsville Pool': "75' x 30' x 8'",
 'Chelsea Pool': "75' x 43' x 10'",
 'Flushing Meadows Corona Park Pool & Rink': "164' x 82' x 7.5'",
 'Gertrude Ederle Pool': "60' x 35' x 8'",
 'Hansborough Pool': "75' x 35' x 9'",
 'Metropolitan Pool': "75'x30'",
 'Recreation Center 54 Pool': "54' x 50' x 12'",
 'Roy Wilkins Pool': "75' x 30' x 9'",
 "St. John's Pool": "75' x 42' x 9'",
 "St. Mary's Pool": "72' x 39' x 9'",
 'Tony Dapolito Pool': "70' x 21' x 8'"}

In [279]:
df = pd.DataFrame.from_dict(pooldimdict.items())

In [281]:
df.columns = ['pool', 'dimensions']

In [283]:
#have to change pool names to match database for easy searching
g = lambda x : 'Asser Levy' if x == 'Asser Levy Pool' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Flushing Meadows Corona Park' if x == 'Flushing Meadows Corona Park Pool & Rink' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Gertrude Ederle' if x == 'Gertrude Ederle Pool' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Brownsville' if x == 'Brownsville Pool' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Chelea' if x == 'Chelsea Pool' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Hansborough' if x == 'Hansborough Pool' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Metropolitan' if x == 'Metropolitan Pool' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'St. Johns' if x == "St. John's Pool" else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Saint Marys' if x == "St. Mary's Pool" else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Tony Dapolito' if x == 'Tony Dapolito Pool' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Recreation Center 54' if x == 'Recreation Center 54 Pool' else x
df['pool'] = df['pool'].map(g)
g = lambda x : 'Roy Wilkins' if x == 'Roy Wilkins Pool' else x
df['pool'] = df['pool'].map(g)


In [328]:
#df #add table to sql database
con = mdb.connect('localhost', 'root', '', 'lap_schedule') 
#indoordim = df.to_sql('indoor_dimensions', con, flavor='mysql')


In [330]:
pooldata = pd.read_sql('SELECT * FROM indoor_dimensions', con)
pooldata

,index,pool,dimensions
0,0,Saint Marys,72' x 39' x 9'
1,1,Tony Dapolito,70' x 21' x 8'
2,2,Metropolitan,75'x30'
3,3,Flushing Meadows Corona Park,164' x 82' x 7.5'
4,4,Gertrude Ederle,60' x 35' x 8'
5,5,Brownsville,75' x 30' x 8'
6,6,Recreation Center 54,54' x 50' x 12'
7,7,St. Johns,75' x 42' x 9'
8,8,Roy Wilkins,75' x 30' x 9'
9,9,Chelea,75' x 43' x 10'


In [335]:
poolsdict = pickle.load(open("other_pickles/poolsandcoords.p", "rb"))

In [339]:
poolsdict
dfpoolcoords = pd.DataFrame.from_dict(poolsdict.items())

In [341]:
dfpoolcoords.columns = ['pools2', 'coords']

In [14]:
indoordimdict = pooldata.set_index('pool')['dimensions'].to_dict()

In [15]:
pickle.dump( indoordimdict, open("indoordimdict.p", "wb" ) )

In [350]:
dimdict = pickle.load(open("other_pickles/indoordimdict.p", "rb"))

In [357]:
dimdict
dimdict.pop("Chelea", None)

"75' x 43' x 10'"

In [359]:
dimdict
pickle.dump( dimdict, open("indoordimdict.p", "wb" ) )

In [362]:
dimdict
dimdict['Gertrude Eredel'] = "60' x 35' x 8'"

In [365]:
dimdict
pickle.dump( dimdict, open("indoordimdict.p", "wb" ) )

In [364]:
dimdict.pop('Gertrude Ederle', None)

"60' x 35' x 8'"

In [352]:
poolsdict

{'Asser Levy': (40.736160099999999, -73.975658199999998),
 'Brownsville': (40.656201800000012, -73.902837200000008),
 'Chelsea': (40.748248699999998, -74.002097599999999),
 'Flushing Meadows Corona Park': (40.751966899999999, -73.836016099999995),
 'Gertrude Eredel': (40.771420700000007, -73.98866439999999),
 'Hansborough': (40.812806599999988, -73.939244799999997),
 'Metropolitan': (40.715018200000003, -73.960390199999992),
 'Recreation Center 54': (40.756490700000001, -73.965117300000003),
 'Roy Wilkins': (40.687243799999997, -73.772132299999996),
 'Saint Marys': (40.811468099999999, -73.914066300000002),
 'St. Johns': (40.673877900000001, -73.934903800000001),
 'Tony Dapolito': (40.729616800000002, -74.005406100000002)}

In [353]:
indoorlist = list(poolsdict.keys())

In [366]:
indoorcombined = {}
for i in indoorlist:
    indoorcombined[i] = (poolsdict[i], dimdict[i])

In [367]:
indoorcombined

{'Asser Levy': ((40.736160099999999, -73.975658199999998), "66' x 26' x 8'"),
 'Brownsville': ((40.656201800000012, -73.902837200000008), "75' x 30' x 8'"),
 'Chelsea': ((40.748248699999998, -74.002097599999999), "75' x 43' x 10'"),
 'Flushing Meadows Corona Park': ((40.751966899999999, -73.836016099999995),
  "164' x 82' x 7.5'"),
 'Gertrude Eredel': ((40.771420700000007, -73.98866439999999),
  "60' x 35' x 8'"),
 'Hansborough': ((40.812806599999988, -73.939244799999997), "75' x 35' x 9'"),
 'Metropolitan': ((40.715018200000003, -73.960390199999992), "75'x30'"),
 'Recreation Center 54': ((40.756490700000001, -73.965117300000003),
  "54' x 50' x 12'"),
 'Roy Wilkins': ((40.687243799999997, -73.772132299999996), "75' x 30' x 9'"),
 'Saint Marys': ((40.811468099999999, -73.914066300000002), "72' x 39' x 9'"),
 'St. Johns': ((40.673877900000001, -73.934903800000001), "75' x 42' x 9'"),
 'Tony Dapolito': ((40.729616800000002, -74.005406100000002),
  "70' x 21' x 8'")}

In [368]:
dfdimcoord=pd.DataFrame(indoorcombined.items(), columns=['pool', 'dimcoord'])
f = lambda x:  x[0][0]
dfdimcoord['latitude']= dfdimcoord['dimcoord'].map(f)

In [370]:
f = lambda x:  x[0][1]
dfdimcoord['longitude']= dfdimcoord['dimcoord'].map(f)

In [371]:
f = lambda x:  x[1]
dfdimcoord['dimensions']= dfdimcoord['dimcoord'].map(f)

In [374]:
del dfdimcoord['dimcoord']

In [375]:
dfdimcoord

,pool,latitude,longitude,dimensions
0,Flushing Meadows Corona Park,40.751967,-73.836016,164' x 82' x 7.5'
1,Recreation Center 54,40.756491,-73.965117,54' x 50' x 12'
2,Metropolitan,40.715018,-73.960390,75'x30'
3,Asser Levy,40.736160,-73.975658,66' x 26' x 8'
4,Roy Wilkins,40.687244,-73.772132,75' x 30' x 9'
5,Tony Dapolito,40.729617,-74.005406,70' x 21' x 8'
6,Saint Marys,40.811468,-73.914066,72' x 39' x 9'
7,St. Johns,40.673878,-73.934904,75' x 42' x 9'
8,Gertrude Eredel,40.771421,-73.988664,60' x 35' x 8'
9,Hansborough,40.812807,-73.939245,75' x 35' x 9'


In [376]:
dfdimcoord.to_csv('indoordimcoord.csv')